In [1]:
import scispacy
import spacy

In [2]:
nlp = spacy.load("en_core_sci_lg")
text = """
Myeloid derived suppressor cells (MDSC) are immature 
myeloid cells with immunosuppressive activity. 
They accumulate in tumor-bearing mice and humans 
with different types of cancer, including hepatocellular 
carcinoma (HCC).
"""
doc = nlp(text)

In [3]:
for s in doc.sents:
    print(s)


Myeloid derived suppressor cells (MDSC) are immature 
myeloid cells with immunosuppressive activity. 

They accumulate in tumor-bearing mice and humans 
with different types of cancer, including hepatocellular 
carcinoma (HCC).



In [4]:
# Examine the entities extracted by the mention detector.
# Note that they don't have types like in SpaCy, and they
# are more general (e.g including verbs) - these are any
# spans which might be an entity in UMLS, a large
# biomedical database.
print(doc.ents)

(Myeloid, suppressor cells, MDSC, immature, myeloid cells, immunosuppressive activity, accumulate, tumor-bearing mice, humans, cancer, hepatocellular 
carcinoma, HCC)


In [5]:
# We can also visualise dependency parses
# (This renders automatically inside a jupyter notebook!):
from spacy import displacy
displacy.render(next(doc.sents), style='dep', jupyter=True)

In [11]:
# AbbreviationDetector

from scispacy.abbreviation import AbbreviationDetector
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

Abbreviation 	 Definition
SBMA 	 (33, 34) Spinal and bulbar muscular atrophy
SBMA 	 (6, 7) Spinal and bulbar muscular atrophy
AR 	 (29, 30) androgen receptor


In [14]:
# UmlsEntityLinker

from scispacy.umls_linking import UmlsEntityLinker
linker = UmlsEntityLinker(resolve_abbreviations=True)

nlp.add_pipe(linker)

doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/tfidf_vectors_sparse.npz not found in cache, downloading to /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmpmm09i4rm
Finished download, copying /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmpmm09i4rm to cache at /Users/willthompson/.scispacy/datasets/ea855fd121a193f03190a91417c209d4cd97e63d3ce4b456c248ef7c13a4ca77.03518aabd12de2103a27a50302f37c3d87b0f313a8be08b5ec306c9c4334b9b1.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/nmslib_index.bin not found in cache, downloading to /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmp9fr__m2s
Finished download, copying /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmp9fr__m2s to cache at /Users/willthompson/.scispacy/datasets/5f620d1bd549a98c005ed601a73806ea2cd1a86ae6c54bbc62bcb3b452ca2630.27a7ac6807fde6628311ff7d70b86fefc640d0eb70637b544c591722a2c16c2a.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scis

/Users/willthompson/anaconda3/envs/textract/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/concept_aliases.json not found in cache, downloading to /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmpdb32fcw9
Finished download, copying /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmpdb32fcw9 to cache at /Users/willthompson/.scispacy/datasets/0f064d20aefab965d5772b2100f8436b3541e7d5313c76cfe5fe070902f149fe.31df9cdb04729860a81bd6c980224ed2bff582586c398d0c9b96ae4e257b9da2.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_2017_aa_cat0129.json not found in cache, downloading to /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmpst5erkjc
Finished download, copying /var/folders/p3/58gq6hbd6r104wvr42fhpbm00000gn/T/tmpst5erkjc to cache at /Users/willthompson/.scispacy/datasets/13b30cd31cd37c1b52f3df6ea023061172d16e9941660e677fdbb29489af7410.4ad71d86ce780e00cab131c7e3b81acfd2f11dd80ccd61125c8bcde506f2ab8a.umls_2017_aa_cat0129.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy

/Users/willthompson/anaconda3/envs/textract/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
entity = doc.ents[1]

print("Name: ", entity)

# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
for umls_ent in entity._.umls_ents:
    print(linker.umls.cui_to_entity[umls_ent[0]])

Name:  bulbar muscular atrophy
CUI: C1839259, Name: Bulbo-Spinal Atrophy, X-Linked
Definition: An X-linked recessive form of spinal muscular atrophy. It is due to a mutation of the gene encoding the ANDROGEN RECEPTOR.
TUI(s): T047
Aliases (abbreviated, total: 50): 
	 Bulbo-Spinal Atrophy, X-Linked, Bulbo-Spinal Atrophy, X-Linked, Atrophies, X-Linked Bulbo-Spinal, Bulbo Spinal Atrophy, X Linked, Bulbo-Spinal Atrophies, X-Linked, X-Linked Bulbo-Spinal Atrophies, Atrophy, X-Linked Bulbo-Spinal, X Linked Bulbo Spinal Atrophy, X-Linked Bulbo-Spinal Atrophy, X-Linked Bulbo-Spinal Atrophy
CUI: C0541794, Name: Skeletal muscle atrophy
Definition: A process, occurring in skeletal muscle, that is characterized by a decrease in protein content, fiber diameter, force production and fatigue resistance in response to different conditions such as starvation, aging and disuse. [GOC:mtg_muscle]
TUI(s): T046
Aliases: (total: 9): 
	 Skeletal muscle atrophy, ATROPHY SKELETAL MUSCLE, skeletal muscle atrophy